# Machine Lerning-based Drug Discovery Pipeline

This notebook presents an end-to-end pipeline for predicting molecular inhibition using multiple molecular representations and machine learning models.

The workflow includes:
1. Data preprocessing
2. Molecular feature extraction
3. Model training and hyperparameter optimization
4. Ensemble learning
5. Predictions and submission file generation

## 1. Environment Setup and Library Import

This section installs and imports all required libraries for molecular processing, graph neural networks, classical machine learning models, and feature extraction.

Key libraries:
- RDKit for molecular representation
- PyTorch Geometric for graph-based learning
- Scikit-learn, XGBoost, LightBGM, CatBoost for regression models

In [ ]:
!pip install rdkit
!pip install torch_geometric
!pip install catboost
!pip install mordred

In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, MACCSkeys, Descriptors3D, QED
from rdkit.DataStructs.cDataStructs import ConvertToNumpyArray
from torch_geometric.data import Data
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.loader import DataLoader
from torch_geometric.data import Batch
from sklearn.ensemble import RandomForestRegressor, StackingRegressor, HistGradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from mordred import Calculator, descriptors

## 2. Data Loading and Preprocessing

Training and test datasets are loaded and cleaned.
Only valid SMILES strings and target values are retained to ensure reliable feature extractions.

In [2]:
# 데이터 불러오기
train = pd.read_csv("train.csv")
train = train[['Canonical_Smiles', 'Inhibition']]
train = train.dropna()

test = pd.read_csv("test.csv")
test = test.dropna()

## 3. Molecular Graph Construction

SMILES strings are converted into graph representations where atoms are nodes and bonds are edges.
This enables graph-based deep learning models.

In [ ]:
def mol_to_graph(mol):
  node_feats = []
  edge_index = []

  for atom in mol.GetAtoms():
    node_feats.append([atom.GetAtomicNum()])

  for bond in mol.GetBonds():
    i = bond.GetBeginAtomIdx()
    j = bond.GetEndAtomIdx()
    edge_index.append((i, j))
    edge_index.append((j, i))

  x = torch.tensor(node_feats, dtype=torch.float)
  edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

  data = Data(x=x, edge_index=edge_index)
  data.num_nodes = x.size(0)
  data.batch = torch.zeros(x.size(0), dtype=torch.long)

  return data

## 4. Graph Neural Network Architecture

A Graph Convolutional Network(GCN) is defiened to learn latent representations of molecules from graph-structed data.

In [ ]:
class GCN(torch.nn.Module):
  def __init__(self, input_dim=1, hidden_dim=64, output_dim=1):
    super().__init__()
    self.conv1 = GCNConv(input_dim, hidden_dim)
    self.conv2 = GCNConv(hidden_dim, hidden_dim)
    self.lin = torch.nn.Linear(hidden_dim, output_dim)

  def forward(self, data):
    x, edge_index, batch = data.x, data.edge_index, data.batch
    x = F.relu(self.conv1(x, edge_index))
    x = F.relu(self.conv2(x, edge_index))
    x = global_mean_pool(x, batch)
    return self.lin(x)

## 5. Training the GNN model

The GNN is trained to predict inhibition values using molecular graph representations.
The trained model is later used as a feature extractor.

In [ ]:
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

graph_data_list = []
for idx, row in train.iterrows():
  mol = Chem.MolFromSmiles(row['Canonical_Smiles'])
  if mol:
    data = mol_to_graph(mol)
    data.y = torch.tensor([row['Inhibition']], dtype=torch.float)
    graph_data_list.append(data)

train_loader = DataLoader(graph_data_list, batch_size=32, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for epoch in range(200):
  for data in train_loader:
    data = data.to(device)
    optimizer.zero_grad()
    out = model(data)
    loss = F.mse_loss(out.squeeze(), data.y)
    loss.backward()
    optimizer.step()

## 6. GNN-based Latent Feature Extraction

Latent molecular representations are extracted from the trained GNN and used as additional features for classical machine learning models.

In [ ]:
def get_gnn_latent(smiles):
  mol = Chem.MolFromSmiles(smiles)
  if mol is None:
    return [0.0] * 1
  data = mol_to_graph(mol)
  data.y = torch.tensor([0.0])
  batch = Batch.from_data_list([data]).to(device)
  model.eval()
  with torch.no_grad():
    out = model(batch)
  return out.view(-1).cpu().tolist()

## 7. Classical Molecular Feature Extraction

This section computes physicochemical descriptors, fingerprints, graph-based statistics, and 3D descriptors for classical ML models.

In [ ]:
def compute_3d_descriptors(smiles):
  mol = Chem.MolFromSmiles(smiles)
  mol = Chem.AddHs(mol)
  success = AllChem.EmbedMolecule(mol, AllChem.ETKDG())
  if success != 0:
    return None
  AllChem.UFFOptimizeMolecule(mol)
  descriptors = {'Asphericity': Descriptors3D.Asphericity(mol), 'Eccentricity': Descriptors3D.Eccentricity(mol), 'InertialShapeFactor': Descriptors3D.InertialShapeFactor(mol), 'SpherocityIndex': Descriptors3D.SpherocityIndex(mol)}
  return descriptors

def compute_graph_features(mol):
  num_atoms = mol.GetNumAtoms()
  num_bonds = mol.GetNumBonds()
  ring_info = mol.GetRingInfo()
  num_rings = ring_info.NumRings()
  distance_matrix = Chem.GetDistanceMatrix(mol)
  diameter = distance_matrix.max()
  return {'NumAtoms': num_atoms, 'NumBonds': num_bonds, 'NumRings': num_rings, 'GraphDiameter': diameter}

calc = Calculator([descriptors.BertzCT])

def extract_features(smiles):
  mol = Chem.MolFromSmiles(smiles)
  if mol is None:
    print(f"Invalid SMILES: {smiles}")
    return None
  try:
      features = [
        Descriptors.MolWt(mol),
        Descriptors.MolLogP(mol),
        Descriptors.NumHAcceptors(mol),
        Descriptors.NumHDonors(mol),
        Descriptors.TPSA(mol),
        Descriptors.NumRotatableBonds(mol),
        Descriptors.FractionCSP3(mol),
        Descriptors.HeavyAtomCount(mol),
        Descriptors.NHOHCount(mol),
        Descriptors.NOCount(mol),
        Descriptors.NumAromaticRings(mol),
        Descriptors.NumSaturatedRings(mol),
        Descriptors.NumAliphaticRings(mol),
        Descriptors.NumAromaticCarbocycles(mol),
        Descriptors.NumValenceElectrons(mol),
        Descriptors.MolMR(mol),
        Descriptors.HallKierAlpha(mol),
        Descriptors.LabuteASA(mol)
        ]
      fp_morgan = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=4096, useFeatures=True, useChirality=True)
      fp_morgan_bits = list(fp_morgan)
      arr_morgan = np.zeros((4096,), dtype=int)
      ConvertToNumpyArray(fp_morgan, arr_morgan)
      fp_morgan_bits = arr_morgan.tolist()

      fp_maccs = MACCSkeys.GenMACCSKeys(mol)
      arr_maccs = np.zeros((167, ), dtype=int)
      ConvertToNumpyArray(fp_maccs, arr_maccs)
      fp_maccs_bits = arr_maccs.tolist()

      graph_feats = compute_graph_features(mol)
      graph_feats_list = list(graph_feats.values())

      desc3d = compute_3d_descriptors(smiles)
      if desc3d is None:
        print(f"Failed to compute 3D descriptors for {smiles}")
        desc3d_list = [0.0] * 4
      else:
        desc3d_list = list(desc3d.values())

      gnn_latent = get_gnn_latent(smiles)

      qed_value = QED.qed(mol)

      res = calc(mol)
      bertz = res['BertzCT']

      return features + fp_morgan_bits + fp_maccs_bits + graph_feats_list + desc3d_list + gnn_latent + [qed_value, bertz]
  except Exception as e:
    print(f"Error processing {smiles}: {e}")
    return None

## 8. Feature Matrix Construction

Multiple molecular features are extracted and combined into tabular datasets for classical machine learning models.
Invalid molecules are excluded during this process.

In [ ]:
# 훈련, 테스트 데이터 생성
X_train_list = []
Y_train_list = []

for smiles, y, in zip(train["Canonical_Smiles"], train["Inhibition"]):
  feats = extract_features(smiles)
  if feats is not None:
    X_train_list.append(feats)
    Y_train_list.append(y)

X_train = pd.DataFrame(X_train_list)
Y_train = pd.Series(Y_train_list, index=X_train.index).astype(float)

X_test_list = []
test_ids = []
for smiles, idx in zip(test["Canonical_Smiles"], test["ID"]):
  feats = extract_features(smiles)
  if feats is not None:
    X_test_list.append(feats)
    test_ids.append(idx)

X_test = pd.DataFrame(X_test_list)

all_features = X_train.columns.tolist()
features_without_bertz = [f for f in X_train.columns if f != 'BertzCT']

## 9. Model Training and Hyperparameter Optimization

Multiple regression models are trained using different feature subsets.
Model-specific feature selection is applied based on empirical performance differences observed during experimentation.

Different models benefited from different feature representations, highliting the importance of model-feature compatibility.

In [ ]:
# 모델 생성 및 적합
xgb_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 6, 7],
    'learning_rate': [0.03, 0.05, 0.07],
    'subsample': [0.7, 0.9, 1.0]
}
xgb_search = RandomizedSearchCV(XGBRegressor(tree_method='hist'), param_distributions=xgb_param_grid, n_iter=30, scoring='neg_mean_squared_error', cv=3, n_jobs=-1)
xgb_search.fit(X_train[features_without_bertz], Y_train)
best_xgb = xgb_search.best_estimator_

rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [15, 20, 25],
    'min_samples_leaf': [5, 10, 15]
}
rf_search = RandomizedSearchCV(RandomForestRegressor(), param_distributions=rf_param_grid, n_iter=30, scoring='neg_mean_squared_error', cv=3, n_jobs=-1)
rf_search.fit(X_train[features_without_bertz], Y_train)
best_rf = rf_search.best_estimator_

lgbm_param_grid = {
    'learning_rate': [0.03, 0.05, 0.07, 0.1],
    'max_depth': [7, 15, 31],
    'lambda_l1': [10, 20, 30],
    'lambda_l2': [10, 20, 30],
    'num_leaves': [20, 30, 40],
    'min_child_samples': [10, 20]
}
lgbm_search = RandomizedSearchCV(LGBMRegressor(), param_distributions=lgbm_param_grid, n_iter=30, scoring='neg_mean_squared_error', cv=3, n_jobs=-1)
lgbm_search.fit(X_train[all_features], Y_train)
best_lgbm = lgbm_search.best_estimator_

cat_param_grid = {
    'learning_rate': [0.03, 0.05, 0.07],
    'depth': [2, 4, 6],
    'l2_leaf_reg': [5, 7, 10],
    'iterations': [200, 400, 600]
}
cat_search = RandomizedSearchCV(CatBoostRegressor(verbose=0, task_type="CPU"), param_distributions=cat_param_grid, n_iter=30, scoring='neg_mean_squared_error', cv=3, n_jobs=-1)
cat_search.fit(X_train[features_without_bertz], Y_train)
best_cat = cat_search.best_estimator_

gradient_param_grid = {
    'max_iter': [300, 400, 500],
    'learning_rate': [0.03, 0.05, 0.07],
    'max_leaf_nodes': [5, 7, 10]
}
gradient_search = RandomizedSearchCV(HistGradientBoostingRegressor(early_stopping=True), param_distributions=gradient_param_grid, n_iter=27, scoring='neg_mean_squared_error', cv=3, n_jobs=-1)
gradient_search.fit(X_train[features_without_bertz], Y_train)
best_gradient = gradient_search.best_estimator_

stacking = StackingRegressor(estimators=[('xgb', best_xgb), ('rf', best_rf), ('cat', best_cat), ('lgbm', best_lgbm)], final_estimator=best_gradient, cv=5, passthrough=False, n_jobs=-1)

stacking.fit(X_train[all_features], Y_train)

## 10. Model Evaluation

The ensemble model is evaluated on the training data using normalized RNSE and Pearson correlation, following the competition evaluation criteria.

In [8]:
# 예측 및 평가
Y_pred_train = stacking.predict(X_train)
Y_pred_test = stacking.predict(X_test)

from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

rmse = np.sqrt(mean_squared_error(Y_train, Y_pred_train))
norm_rmse = rmse / (max(Y_train) - min(Y_train))
corr, _ = pearsonr(Y_train, Y_pred_train)
score = 0.5 * (1- min(norm_rmse, 1)) + 0.5 * corr

print(f"Normalized RMSE: {norm_rmse:.4f}")
print(f"Pearson Correlation: {corr:.4f}")
print(f"Final Score: {score:4f}")

[LightGBM] [Warning] lambda_l2 is set=10, reg_lambda=0.0 will be ignored. Current value: lambda_l2=10
[LightGBM] [Warning] lambda_l1 is set=10, reg_alpha=0.0 will be ignored. Current value: lambda_l1=10
[LightGBM] [Warning] lambda_l2 is set=10, reg_lambda=0.0 will be ignored. Current value: lambda_l2=10
[LightGBM] [Warning] lambda_l1 is set=10, reg_alpha=0.0 will be ignored. Current value: lambda_l1=10
Normalized RMSE: 0.1938
Pearson Correlation: 0.7500
Final Score: 0.778092


## 11. Submission File Generation

Final predictions on the test dataset are generated and saved in the required submission format.

In [9]:
# 파일 작성
submission = pd.DataFrame({"ID": test_ids, "Inhibition": Y_pred_test})
submission.to_csv("submission_250729_1.csv", index=False)